# Day 27: Machine Super Intelligence

> Shane Legg (2008) — [Machine Super Intelligence](https://www.vetta.org/documents/Machine_Super_Intelligence.pdf)

Welcome to Day 27. Today we transition from the mathematical theory of information (Day 26) to the mathematical theory of *intelligence*. 

Shane Legg's thesis formalizes the idea that intelligence is not about being good at a specific game (like Chess), but about being a **Generalist** who can adapt to any reward-summarizing environment.

## What We'll Do
1. **The formal definition**: Implementing $\Upsilon(\pi)$.
2. **Occam's weighting**: Visualizing why $2^{-K}$ is the 'Gold Standard' for general intelligence.
3. **Agent Showdown**: Comparing Random vs. Specialized RL vs. Compression-based Agents.
4. **Simulation**: Proving the Invariance Theorem.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from implementation import (
    UniversalIntelligenceMeasure, 
    GridWorld, 
    PatternSequence, 
    RandomAgent, 
    SimpleRLAgent, 
    PredictiveAgent
)

%matplotlib inline
np.random.seed(42)
print("Framework loaded. Ready for evaluation.")

## 1. The Intelligence Formula

Shane Legg defines $\Upsilon(\pi)$ as the weighted average of performance over all computable environments $\mu$.

$$\Upsilon(\pi) = \sum_{\mu \in E} 2^{-K(\mu)} V_{\mu}^\pi$$

Let's visualize the **Universal Weighting** ($2^{-K}$). This is why 'Simple' matters more than 'Noisy'.

In [ ]:
complexities = np.arange(1, 11)
weights = 2.0**(-complexities)

plt.figure(figsize=(8, 4))
plt.bar(complexities, weights, color='salmon', alpha=0.7)
plt.yscale('log')
plt.title("The Universal Prior (2^-K)")
plt.xlabel("Environment Complexity K")
plt.ylabel("Weight (Log Scale)")
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

print(f"Note: An environment of complexity 10 is {2**8}x less important than one of complexity 2.")

## 2. Defining the Test Suite

We use a mix of GridWorlds (navigation) and PatternSequences (induction) as our proxy for the space $E$.

In [ ]:
envs = [
    GridWorld(size=3),        # K=3 (Simple navigation)
    GridWorld(size=8),        # K=8 (Medium navigation)
    PatternSequence([0, 1]),  # K=4 (Simple induction)
    PatternSequence([0, 1, 2, 3, 0, 1, 2]), # K=14 (Complex pattern)
]

measure = UniversalIntelligenceMeasure(envs)
print(f"Test suite initialized with {len(envs)} environments.")

## 3. The Benchmark Showdown

We compare three different philosophies of intelligence:
1. **Baseline**: Acts randomly.
2. **Simple RL (Q-Table)**: Specialized learner.
3. **Predictive Agent**: Acts as a proxy for Solomonoff Induction (Compression-based).

In [ ]:
agents = {
    "Random": RandomAgent(),
    "Simple RL": SimpleRLAgent(),
    "Predictive": PredictiveAgent()
}

scores = {}
for name, agent in agents.items():
    print(f"Evaluating {name}...")
    # We give them a generous 50 episodes to show their learning capacity
    res = measure.evaluate(agent, episodes=50)
    scores[name] = res['upsilon_normalized']
    print(f"  {name} Upsilon: {scores[name]:.4f}")

## 4. Discussion: Why did the Predictive Agent win?

In Legg's theory, the core of intelligence is **Generalization**. 

- The **Simple RL** agent tries to maximize reward through trial and error.
- The **Predictive Agent** tries to 'compress' the environment's history into a map. 

Because many environments share underlying logical structures (cycles, grids), the ability to build an internal model (compression) allows the Predictive agent to solve new environments faster than a pure tabula-rasa RL agent.

## Key Takeaways

1. **Non-Anthropocentric**: Upsilon doesn't care if you're a human or a computer. It only cares if you achieve goals.
2. **Simple is Universal**: The weight $2^{-K}$ formalizes Occam's Razor as a fundamental law of intelligence.
3. **The Limit**: AIXI is the 'Perfect' agent with maximum Upsilon, but it is non-computable because checking all possible programs takes infinite time.